# W261 Spring - Homework 4


## HW 4.0

**What is MrJob? How is it different to Hadoop MapReduce? **

MRJob is a Python framework to make running complex Map Reduce tasks much simpler. It is capable of running sequences of MapReduce or even iterative MapReduce jobs. The really nice thing about MRJob is the almost pseudo-code like way of expressing how to execute and combine MapReduce jobs.

MRJob is not Hadoop but it can execute in a stand-alone mode to run your MapReduce jobs, useful for small scale testing. MRJob also can submit your job to Hadoop via the Streaming API, whether on a local or remote Hadoop cluster. In addition, MRJob has a very nice integration with Amazon AWS Elastic Map Reduce, allowing the researcher to focus on the MapReduce and analysis instead of the infrastructure on which to execute it.

**What are the mapper_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?**

MRJob defines a base class that you as the developer must override to use MRJob. The base class executes the mapper, reducer, and combiner functions when you override them in the class. The MRJob base class also provides intializer and finalizer methods for each of the mapper, combiner and reducer functions. These methods are `mapper_init()`, `combiner_init()`, `reducer_init()`, `mapper_final()`, `combiner_final()`, and `reducer_final()` respectively. The init() methods are called before the corresponding `mapper()`, `combiner()`, `reducer()` methods, allowing setup of data or other things before the method is called. The final() methods are called immediately after the `mapper()`, `reducer()` or `combiner()` methods.

## HW 4.1

**What is serialization in the context of MrJob or Hadoop?**

Serialization is the process of converting a machine representation of an object to a format used for storage or transmission. In the context of Hadoop Streaming all input and output is treated as a character stream with keys and values separated by tabs (or another specified delimiter). In the case of MRJob, serialization consists of three types: raw, json, or pickle. Raw is text streams, json is json formatted text streams, and pickle is the Python binary serialization method.

**When it used in these frameworks?**

MRJob uses serialization for input and output as well as internal transmission of objects. Each place serialization is used can be defined by the type of protocol.

**What is the default serialization mode for input and outputs for MrJob? **

The default serialization mode for MRJob inputs is `RAWValueProtocol` which reads lines of text with no key - it's just a stream of text. The default output protocol is `JSONprotocol` which outputs JSON formatted strings separated by a tab character.

## HW 4.2: 

Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html<br/>
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/<br/>

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:

    C,"10001",10001   #Visitor id 10001
    V,1000,1          #Visit by Visitor 10001 to page id 1000
    V,1001,1          #Visit by Visitor 10001 to page id 1001
    V,1002,1          #Visit by Visitor 10001 to page id 1002
    C,"10002",10002   #Visitor id 10001

V
Note: #denotes comments to the format:

    V,1000,1,C, 10001
    V,1001,1,C, 10001
    V,1002,1,C, 10001

Write the python code to accomplish this.

In [1]:
%%writefile msdata_transform.py
# HW 4.2
#
# Read a CSV file that contains page visits by customers
# A customer id record is followed by a number of page id records which are the pages
# the customer visited on the web site
# Transform the data such that the page visits contain the customer ids on the same record
# The result is the elimination of the standalone customer id record and an extended page
# visit record containing the page id and customer id
# The file contains other records which are output unmodified
#
import sys
with open('anonymous-msweb.data','rU') as datafile:
    
    # iterate over the lines in the data file
    for line in datafile.readlines():
        
        # split the line into the CSV fields (tokens)
        tokens = line.strip().split(',')
        
        # if a customer record then retain the customer id
        if tokens[0] == 'C':
            customer_id = tokens[2]
            
        # if a page visit record then transform to append the customer id
        # V,pageid,count,C,cust_id
        elif tokens[0] == 'V':
            sys.stdout.write('{0},{1},{2},{3},{4}\n'.format('V',tokens[1],tokens[2],'C',customer_id))
            
        # otherwise just output the line
        else:
            sys.stdout.write(line)

Writing msdata_transform.py


In [2]:
!python msdata_transform.py >anonymous-msweb-transformed.data

## HW 4.3: 

Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).

In [15]:
%%writefile mostvisitedpage.py
from mrjob.job import MRJob 
from mrjob.step import MRStep
import heapq

class MRMostVisitedPage(MRJob):
    def mapper_get_visits(self, _, record):
        self.increment_counter('Execution Counts', 'mapper calls', 1)
        # yield each visit in the line
        tokens = record.split(',')
        if tokens[0] == 'V':
            yield (tokens[1], 1)

    def combiner_count_visits(self, page, counts): 
        self.increment_counter('Execution Counts', 'combiner calls', 1)
        # sum the page visits we've seen so far
        yield (page, sum(counts))
        
    def reducer_count_visits(self, page, counts):
        self.increment_counter('Execution Counts', 'reducer_count calls', 1)
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function. yield None, (sum(counts), page)
        # discard the key; it is just None
        yield None, (sum(counts), page)
        
    def reducer_find_top5_visits(self, _, page_count_pairs):
        self.increment_counter('Execution Counts', 'reducer_find_max calls', 1)
        # each item of page_count_pairs is (count, page),
        # so yielding one results in key=counts, value=page yield max(page_count_pairs)
        return heapq.nlargest(5, page_count_pairs)

        
    def steps(self): return [
            MRStep(mapper=self.mapper_get_visits,
                   combiner=self.combiner_count_visits,
                   reducer=self.reducer_count_visits),
            MRStep(reducer=self.reducer_find_top5_visits)
        ]
    
if __name__ == '__main__': 
    MRMostVisitedPage.run()

Overwriting mostvisitedpage.py


In [16]:
!python mostvisitedpage.py anonymous-msweb-transformed.data

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostvisitedpage.rcordell.20160208.015734.738794
writing to /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostvisitedpage.rcordell.20160208.015734.738794/step-0-mapper_part-00000
Counters from step 1:
  Execution Counts:
    combiner calls: 285
    mapper calls: 98955
writing to /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostvisitedpage.rcordell.20160208.015734.738794/step-0-mapper-sorted
> sort /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostvisitedpage.rcordell.20160208.015734.738794/step-0-mapper_part-00000
writing to /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostvisitedpage.rcordell.20160208.015734.738794/step-0-reducer_part-00000
Counters from step 1:
  Execution Counts:
    combiner calls: 285
    mapper calls: 98955
    reducer_count calls: 285
writing to /var/folders/z_/rfp5q2cd6db13

## HW 4.4: 

Find the most frequent visitor of each page using MrJob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

In [167]:
%%writefile mostfreqvisitors.py
from mrjob.job import MRJob 
from mrjob.step import MRStep 

class MRMostFrequentVisitors(MRJob):
    def configure_options(self):
        super(MRMostFrequentVisitors, self).configure_options()
        self.SORT_VALUES = True
        
    # generate a dictionary of pages and URLs for them
    def mapper_get_visits_init(self):
        # create a dictionary to use for the page URLs and ids
        self.pages = {}
        
    # generate keys of page,customer,url and values of 1
    def mapper_get_visits(self, _, record):
        self.increment_counter('Execution Counts', 'mapper calls', 1)
        tokens = record.split(',')
        
        # the page definitions come first in the file so create a dictionary from them.
        if tokens[0] == 'A':
            self.pages[tokens[1]] = tokens[4].strip('"')
            
        # emit a key = (page_id, client_id, url) and value = 1
        elif tokens[0] == 'V':
            yield ((tokens[1], tokens[4], self.pages[tokens[1]]), 1)
        else:
            pass

    # combine page visits by key where the key is page,customer
    def combiner_count_visits(self, key, counts): 
        self.increment_counter('Execution Counts', 'combiner count visits', 1)
        # sum the keys we've seen so far.
        # the key is (page_id, cust_id, page_url) so we're counting page views by client
        yield (key, sum(counts))
        
    # set up instance variables to use to calculate the max visits to a page by a single customer
    def reducer_count_visits_init(self):
        self.current_page = None
        self.max_count = 0
        
    # count the visits per page per customer and also compute the max visits per page by a single customer
    def reducer_count_visits(self, key, counts):
        self.increment_counter('Execution Counts', 'reducer_count visits', 1)
        # make sure we have sums of all keys 
        s = sum(counts)
        if self.current_page == key[0]:
            if self.max_count < s:
                self.max_count = s
        else:
            if self.current_page:
                p = self.current_page
                t = self.max_count
                yield((self.current_page,'*',key[2]), t)
                
            self.current_page = key[0]
            self.max_count = s

        yield (key, s)

    # set up a variable to contain the current page max count value
    def reducer_find_max_visits_init(self):
        self.page_max = 0
     
    # yield the max visits to a page and the customers that made them
    def reducer_find_max_visits(self, key, counts):
        self.increment_counter('Execution Counts', 'reducer_find_max visits', 1)
        
        # if this is the key with the max visits for the page then stash it
        if key[1] == '*':
            self.page_max = sum(counts)
        else:
            # otherwise sum the counts and store a local copy because it exhausts the generator
            p = sum(counts)
            # if this count is the same as the max visits for the page, yield it
            if p == self.page_max:
                yield key, p
        
          
    def steps(self): return [
            MRStep(mapper_init=self.mapper_get_visits_init,
                    mapper=self.mapper_get_visits,
                   combiner=self.combiner_count_visits,
                   reducer_init=self.reducer_count_visits_init,
                   reducer=self.reducer_count_visits),
            MRStep(reducer_init=self.reducer_find_max_visits_init,
                    reducer=self.reducer_find_max_visits)
        ]
    
if __name__ == '__main__': 
    MRMostFrequentVisitors.run()

Overwriting mostfreqvisitors.py


In [169]:
!python mostfreqvisitors.py anonymous-msweb-transformed.data > max_page_visits_customer.output

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
creating tmp directory /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostfreqvisitors.rcordell.20160208.053219.800269
writing to /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostfreqvisitors.rcordell.20160208.053219.800269/step-0-mapper_part-00000
Counters from step 1:
  Execution Counts:
    combiner count visits: 98654
    mapper calls: 98955
writing to /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostfreqvisitors.rcordell.20160208.053219.800269/step-0-mapper-sorted
> sort /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostfreqvisitors.rcordell.20160208.053219.800269/step-0-mapper_part-00000
writing to /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/mostfreqvisitors.rcordell.20160208.053219.800269/step-0-reducer_part-00000
Counters from step 1:
  Executio

In [170]:
!cat max_page_visits_customer.output | head -100

["1000", "10001", "/regwiz"]	1
["1000", "10010", "/regwiz"]	1
["1000", "10039", "/regwiz"]	1
["1000", "10073", "/regwiz"]	1
["1000", "10087", "/regwiz"]	1
["1000", "10101", "/regwiz"]	1
["1000", "10132", "/regwiz"]	1
["1000", "10141", "/regwiz"]	1
["1000", "10154", "/regwiz"]	1
["1000", "10162", "/regwiz"]	1
["1000", "10166", "/regwiz"]	1
["1000", "10201", "/regwiz"]	1
["1000", "10218", "/regwiz"]	1
["1000", "10220", "/regwiz"]	1
["1000", "10324", "/regwiz"]	1
["1000", "10348", "/regwiz"]	1
["1000", "10376", "/regwiz"]	1
["1000", "10384", "/regwiz"]	1
["1000", "10409", "/regwiz"]	1
["1000", "10429", "/regwiz"]	1
["1000", "10454", "/regwiz"]	1
["1000", "10457", "/regwiz"]	1
["1000", "10471", "/regwiz"]	1
["1000", "10497", "/regwiz"]	1
["1000", "10511", "/regwiz"]	1
["1000", "10520", "/regwiz"]	1
["1000", "10541", "/regwiz"]	1
["1000", "10564", "/regwiz"]	1
["1000", "10599", "/regwiz"]	1
["1000", "10752", "/regwiz"]	1
["1000", "10756", "/regwiz"]	1
["1000", "10861", "/regwiz"]	1
["1000",

In [171]:
!cat max_page_visits_customer.output | tail -100

["1295", "38244", "/train_cert"]	1
["1295", "38296", "/train_cert"]	1
["1295", "38313", "/train_cert"]	1
["1295", "38454", "/train_cert"]	1
["1295", "38571", "/train_cert"]	1
["1295", "38573", "/train_cert"]	1
["1295", "38661", "/train_cert"]	1
["1295", "38678", "/train_cert"]	1
["1295", "38755", "/train_cert"]	1
["1295", "38831", "/train_cert"]	1
["1295", "38869", "/train_cert"]	1
["1295", "38953", "/train_cert"]	1
["1295", "38981", "/train_cert"]	1
["1295", "38998", "/train_cert"]	1
["1295", "39024", "/train_cert"]	1
["1295", "39033", "/train_cert"]	1
["1295", "39058", "/train_cert"]	1
["1295", "39066", "/train_cert"]	1
["1295", "39094", "/train_cert"]	1
["1295", "39105", "/train_cert"]	1
["1295", "39112", "/train_cert"]	1
["1295", "39131", "/train_cert"]	1
["1295", "39194", "/train_cert"]	1
["1295", "39221", "/train_cert"]	1
["1295", "39284", "/train_cert"]	1
["1295", "39293", "/train_cert"]	1
["1295", "39493", "/train_cert"]	1
["1295", "39505", "/train_cert"]	1
["1295", "39550", "/

## HW 4.5 Clustering Tweet Dataset

Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

- 0: Human, where only basic human-human communication is observed.

- 1: Cyborg, where language is primarily borrowed from other sources
    (e.g., jobs listings, classifieds postings, advertisements, etc...).

- 2: Robot, where language is formulaically derived from unrelated sources
    (e.g., weather/seismology, police/fire event logs, etc...).

- 3: Spammer, where language is replicated to high multiplicity
    (e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of  recent research, which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

    topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

    USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
    .
    .

    where

    USERID = unique user identifier
    CODE = 0/1/2/3 class code
    TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

- (A) K=4 uniform random centroid-distributions over the 1000 words (generate 1000 random numbers and normalize the vectors)
- (B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
- (C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
- (D) K=4 "trained" centroids, determined by the sums across the classes. Use use the (row-normalized) class-level aggregates as 'trained' starting centroids (i.e., the training is already done for you!).

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

    topUsers_Apr-Jul_2014_1000-words_summaries.txt

    Row 1: Words
    Row 2: Aggregated distribution across all classes
    Row 3-6 class-aggregated distributions for clases 0-3
    
- For (A),  we select 4 users randomly from a uniform distribution [1,...,1,000]
- For (B), (C), and (D)  you will have to use data from the auxiliary file: 

    `topUsers_Apr-Jul_2014_1000-words_summaries.txt`

This file contains 5 special word-frequency distributions:

- (1) The 1000-user-wide aggregate, which you will perturb for initializations in parts (B) and (C), and
- (2-5) The 4 class-level aggregates for each of the user-type classes (0/1/2/3)

In parts (B) and (C), you will have to perturb the 1000-user aggregate 
(after initially normalizing by its sum, which is also provided).
So if in (B) you want to create 2 perturbations of the aggregate, start
with (1), normalize, and generate 1000 random numbers uniformly 
from the unit interval (0,1) twice (for two centroids), using:

from numpy import random
numbers = random.sample(1000)

Take these 1000 numbers and add them (component-wise) to the 1000-user aggregate,
and then renormalize to obtain one of your aggregate-perturbed initial centroids.

    ###################################################################################
    # Generate random initial centroids around the global aggregate
    # Part (B) and (C) of this question
    ###################################################################################
    def startCentroidsBC(k):
        counter = 0
        for line in open("topUsers_Apr-Jul_2014_1000-words_summaries.txt").readlines():
            if counter == 2:        
                data = re.split(",",line)
                globalAggregate = [float(data[i+3])/float(data[2]) for i in range(1000)]
            counter += 1
        ## perturb the global aggregate for the four initializations    
        centroids = []
        for i in range(k):
            rndpoints = random.sample(1000)
            peturpoints = [rndpoints[n]/10+globalAggregate[n] for n in range(1000)]
            centroids.append(peturpoints)
            total = 0
            for j in range(len(centroids[i])):
                total += centroids[i][j]
            for j in range(len(centroids[i])):
                centroids[i][j] = centroids[i][j]/total
        return centroids

------
For experiments A, B, C and D and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.



In [1]:
%load_ext autoreload
%autoreload 2

In [147]:
%%writefile assigncluster.py
from mrjob.job import MRJob 
from mrjob.step import MRStep 
from numpy import array, argmin

#Calculate find the nearest centroid for data point 
def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    centroid_points = array(centroid_points)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    return minidx

class AssignCluster(MRJob):
    centroid_points=[]
    centroids_file = '/Users/rcordell/Documents/MIDS/W261/week04/HW4/centroids.txt'
    
    def steps(self):
        return [
            MRStep(mapper_init = self.cluster_mapper_init, 
                      mapper=self.cluster_mapper)
        ]
    
    #load centroids info from file
    def cluster_mapper_init(self):
        self.centroid_points = [map(float,s.split('\n')[0].split(',')) for s in open(self.centroids_file).readlines()]
        
    #load data and output the cluster id and customer id 
    def cluster_mapper(self, _, line):
        self.increment_counter('Execution Counts', 'mapper', 1)
        values = (map(float,line.split(',')))
        cust_id = int(values[0])
        vector = [x/values[2] for x in values[3:]]
        yield int(MinDist(vector,self.centroid_points)), cust_id

    
if __name__ == '__main__': 
    AssignCluster.run()

Overwriting assigncluster.py


In [148]:
%%writefile mrkmeans.py
from numpy import argmin, array, random, zeros
from mrjob.job import MRJob 
from mrjob.step import MRStep 

#Calculate find the nearest centroid for data point 
def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    centroid_points = array(centroid_points)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    return minidx

class MRKMeans(MRJob):
    centroid_points=[]
    k=4 
    centroids_file = '/Users/rcordell/Documents/MIDS/W261/week04/HW4/centroids.txt'
    def steps(self):
        return [
            MRStep(mapper_init = self.mapper_init, 
                      mapper=self.mapper,
                      combiner = self.combiner,
                      reducer=self.reducer)
        ]
    
    #load centroids info from file
    def mapper_init(self):
        self.centroid_points = [map(float,s.split('\n')[0].split(',')) for s in open(self.centroids_file).readlines()]
        open(self.centroids_file, 'w').close()
        
    #load data and output the nearest centroid index and data point 
    def mapper(self, _, line):
        self.increment_counter('Execution Counts', 'mapper', 1)
        values = (map(float,line.split(',')))
        vector = [x/values[2] for x in values[3:]]
        yield int(MinDist(vector,self.centroid_points)), (vector,1)
        
    #Combine sum of data points locally
    def combiner(self, cluster, inputdata):
        self.increment_counter('Execution Counts', 'combiner', 1)
        vector_sum = [0]*1000
        num = 0
        for vector, n in inputdata:
            num = num + n
            vector_sum = [sum(x) for x in zip(vector_sum, vector)]
        yield cluster,(vector_sum,num)
        
    #Aggregate sum for each cluster and then calculate the new centroids
    def reducer(self, cluster, inputdata): 
        self.increment_counter('Execution Counts', 'reducer', 1)
        centroids = []
        num = 0.0 
        vector = [0.0]*1000
        for vector_sum, n in inputdata:
            num = num + n
            vector = [sum(x) for x in zip(vector, vector_sum)]
        
        # the new centroids are the means of the sums of all the member vector elements
        vector = [x/num for x in vector]

        yield cluster,vector
    
if __name__ == '__main__': 
    MRKMeans.run()

Overwriting mrkmeans.py


MRJob Driver

In [173]:
# initialize centroids by choosing k vectors at random from the data set
from numpy import random, array, shape
from mrkmeans import MRKMeans
from assigncluster import AssignCluster
from itertools import chain

k = 4
max_iterations = 10
datafile = '/Users/rcordell/Documents/MIDS/W261/week04/HW4/topUsers_Apr-Jul_2014_1000-words.txt'
centroidsfile = '/Users/rcordell/Documents/MIDS/W261/week04/HW4/centroids.txt'
scoresfile = '/Users/rcordell/Documents/MIDS/W261/week04/HW4/scores.txt'
summaryfile = '/Users/rcordell/Documents/MIDS/W261/week04/HW4/topUsers_Apr-Jul_2014_1000-words_summaries.txt'

scores = {}

#Check whether centroids converge
def stop_criterion(centroid_points_old, centroid_points_new, threshold):
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    Diff = [abs(x-y) for x, y in zip(oldvalue, newvalue)]
    Flag = True
    for element in Diff:
        if(element > threshold):
            Flag = False
            break
    return Flag

def persist_centroids(centroids_filename, centroids):
    # persist the centroids to disk 
    with open(centroids_filename,'w+') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroids)

def initialize_centroids(k, datafile):
    centroids = []
    # generate an array of k randomly chosen integers.
    seed = sorted(random.random_integers(1, 1000, (1, k))[0])


    # select the vectors from the data file from the randomly generated integers
    with open(datafile, 'rU') as infile:
        for i, line in enumerate(infile):
            # go to the ith vector in the file
            if i+1 in seed:
                # read the line which is userid,code,total,word1 count, word2 count,... , word1000 count
                values = [float(x) for x in line.split(',')]

                # normalize by dividing the word counts by the total word count
                centroids.append(array([x/values[2] for x in values[3:]]))

    return centroids    


def initialize_centroids_perturb(k, summaryfile):
    with open(summaryfile) as infile:
        for i, line in enumerate(infile):
            # second line is the aggregrated vector
            if i == 1:        
                data = line.split(",")
                globalAggregate = [float(data[i+3])/float(data[2]) for i in range(1000)]
    ## perturb the global aggregate for the four initializations    
    centroids = []
    for i in range(k):
        rndpoints = random.sample(1000)
        
        # why do we divide the randomized vector by 10 before adding to the aggregate vector?
        peturpoints = [rndpoints[n]/10+globalAggregate[n] for n in range(1000)]
        centroids.append(peturpoints)
        total = 0
        for j in range(len(centroids[i])):
            total += centroids[i][j]
        for j in range(len(centroids[i])):
            centroids[i][j] = centroids[i][j]/total
    return centroids  

def initialize_centroids_trained(k, summaryfile):
    centroids = []
    with open(summaryfile) as infile:
        for i, line in enumerate(infile):
            if i > 1:        
                data = line.split(",")
                centroids.append([float(data[i+3])/float(data[2]) for i in range(1000)])

    return centroids

if __name__ == '__main__':
    
#    centroids = initialize_centroids_trained(k, summaryfile)    
    centroids = initialize_centroids(k, datafile)
#    centroids = initialize_centroids_perturb(k, summaryfile)
    
    persist_centroids(centroidsfile, centroids)
    mr_job = MRKMeans(args=[datafile,'--strict-protocols'])
    assign_cluster = AssignCluster(args=[datafile,'--strict-protocols'])

    # Update centroids iteratively
    for i in range(max_iterations):
        # save previous centoids to check convergency
        centroids_old = centroids[:]

        print "iteration"+str(i)+":"

        # Assign the customers to a cluster
        with assign_cluster.make_runner() as runner:
            runner.run()

            for line in runner.stream_output():
                cluster,cust_id =  assign_cluster.parse_output_line(line)
                scores[cust_id] = cluster

        # write out the customer classifications/cluster assignments
        with open(scoresfile, 'a') as s:
            for cust_id in scores:
                s.write('{0},{1}\n'.format(cust_id, scores[cust_id]))

        # update the centroids
        with mr_job.make_runner() as runner: 
            runner.run()

            # stream_output: get access of the output 
            for line in runner.stream_output():
                cluster,centroid =  mr_job.parse_output_line(line)
                centroids[cluster] = centroid     

        with open(centroidsfile, 'a') as f:
            f.writelines(','.join(str(element) for element in centroid) + '\n' for centroid in centroids)

        if(stop_criterion(centroids_old,centroids,0.01)):
            break
            
    # compare our classification to the training set
    train_Y = {}
    tally = {}
    with open(datafile, 'rU') as infile:
        for line in infile.readlines():
            data = line.split(",")
            train_Y[int(data[0])] = int(data[1])
    for cust_id in train_Y:
        if train_Y[cust_id] not in tally:
            tally[train_Y[cust_id]] = {'right' : 0, 'wrong' : 0}
        if train_Y[cust_id] == scores[cust_id]:
            tally[train_Y[cust_id]]['right'] += 1
        else:
            tally[train_Y[cust_id]]['wrong'] += 1

    print tally
    
    

iteration0:
iteration1:
iteration2:
iteration3:
{0: {'wrong': 734, 'right': 18}, 1: {'wrong': 90, 'right': 1}, 2: {'wrong': 33, 'right': 21}, 3: {'wrong': 53, 'right': 50}}


## HW4.6  (OPTIONAL) Scaleable K-MEANS++ 

Over half a century old and showing no signs of aging,
k-means remains one of the most popular data processing
algorithms. As is well-known, a proper initialization
of k-means is crucial for obtaining a good final solution.
The recently proposed k-means++ initialization algorithm
achieves this, obtaining an initial set of centers that is provably
close to the optimum solution. A major downside of the
k-means++ is its inherent sequential nature, which limits its
applicability to massive data: one must make k passes over
the data to find a good initial set of centers. The paper listed below 
shows how to drastically reduce the number of passes needed
to obtain, in parallel, a good initialization. This is unlike
prevailing efforts on parallelizing k-means that have mostly
focused on the post-initialization phases of k-means. The 
proposed initialization algorithm k-means||
obtains a nearly optimal solution after a logarithmic number
of passes; the paper also shows that in practice a constant
number of passes suffices. Experimental evaluation on realworld
large-scale data demonstrates that k-means|| outperforms
k-means++ in both sequential and parallel settings.

Read the following paper entitled "Scaleable K-MEANS++" located at:

http://theory.stanford.edu/~sergei/papers/vldb12-kmpar.pdf 

In MrJob, implement K-MEANS|| and compare with a random initializtion when used in 
conjunction with the kmeans algorithm as an initialization step for the 2D  dataset 
generated using code in the following notebook:

https://www.dropbox.com/s/lbzwmyv0d8rocfq/MrJobKmeans.ipynb?dl=0

Plot the initialation centroids and the centroid trajectory as the K-MEANS|| algorithms iterates. 
Repeat this for a random initalization (i.e., pick a training vector at random for each inital centroid)
of the kmeans algorithm. Comment on the trajectories of both algorithms.
Report on the number passes over the training data, and time required to run both  clustering algorithms.
Also report the rand index score for both algorithms and comment on your findings.


4.6.1 Apply your implementation of K-MEANS|| to the dataset  in HW 4.5 and compare to the a 
random initalization (i.e., pick a training vector at random for each inital centroid)of the kmeans algorithm.
Report on the number passes over the training data, and time required to run all  clustering algorithms. 
Also report the rand index score for both algorithms and comment on your findings.

## HW4.7   (OPTIONAL) Canopy Clustering

An alternative way to intialize the k-means algorithm is the  canopy clustering. The canopy clustering 
algorithm is an unsupervised pre-clustering algorithm introduced by Andrew McCallum, Kamal Nigam and 
Lyle Ungar in 2000. It is often used as preprocessing step for the K-means algorithm or the 
Hierarchical clustering algorithm. It is intended to speed up clustering operations on large data sets, 
where using another algorithm directly may be impractical due to the size of the data set.

For more details on the Canopy Clustering algorithm see:

https://en.wikipedia.org/wiki/Canopy_clustering_algorithm

Plot the initialation centroids and the centroid trajectory as the Canopy Clustering based K-MEANS algorithm iterates. 
Repeat this for a random initalization (i.e., pick a training vector at random for each inital centroid)
of the kmeans algorithm. Comment on the trajectories of both algorithms.
Report on the number passes over the training data, and time required to run both  clustering algorithms.
Also report the rand index score for both algorithms and comment on your findings.

4.7.1 Apply your implementation Canopy Clustering based K-MEANS algorithm to the dataset  in HW 4.5 and compare to the a 
random initalization (i.e., pick a training vector at random for each inital centroid)of the kmeans algorithm.
Report on the number passes over the training data, and time required to run both  clustering algorithms. 
Also report the rand index score for both algorithms and comment on your findings.